<a target="_blank" href="https://colab.research.google.com/github/Xornotor/PPGEE-Otimizacao-Exercicios/blob/main/Lista-01-A/Q6.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# **Lista de Exercícios 01-A | Questão 6**

**UFBA** | PPGEE0016 - Otimização

**Aluno:** André Paiva Conrado Rodrigues

In [1]:
# Importação de dependências
import numpy as np
from scipy.optimize import linprog

## **1. Problema de Otimização**

\begin{equation*}
\begin{aligned}
\text{Maximizar} \\
&& Z = 5x_1 + 4x_2 + 3x_3 \\
\text{Sujeito a} \\
c_{1}: && 2x_1 + 3x_2 + x_3 \leq 5 \\
c_{2}: && 4x_1 + 2x_2 + 2x_3 \leq 11 \\
c_{3}: && 3x_1 + 2x_2 + 2x_3 \leq 8 \\
&& x_1, x_2, x_3 \geq 0 \\
\end{aligned}
\end{equation*}

## **2. Solução manual por Simplex**

Primeiramente, é necessário colocar a função objetivo e as restrições na forma padrão, adicionando variáveis de folga para eliminar as desigualdades. Deste modo, o problema é reformulado da seguinte maneira:

\begin{equation*}
\begin{aligned}
\text{Maximizar} \\
&& Z - 5x_1 - 4x_2 - 3x_3 = 0 \\
\text{Sujeito a} \\
c_{1}: && 2x_1 + 3x_2 + x_3 + x_4 = 5 \\
c_{2}: && 4x_1 + 2x_2 + 2x_3 + x_5 = 11 \\
c_{3}: && 3x_1 + 2x_2 + 2x_3 + x_6 = 8 \\
&& x_1, x_2, x_3, x_4, x_5, x_6 \geq 0 \\
\end{aligned}
\end{equation*}

Montando a matriz $\boldsymbol{M}$ para o algoritmo Simplex:

|        ////        | $Z$ | $x_1$ | $x_2$ | $x_3$ | $x_4$ | $x_5$ | $x_6$ |  $b$  |
|--------------------|-----|-------|-------|-------|-------|-------|-------|-------|
| $\boldsymbol{L_1}$ |  0  |   2   |   3   |   1   |   1   |   0   |   0   |   5   |
| $\boldsymbol{L_2}$ |  0  |   4   |   2   |   2   |   0   |   1   |   0   |   11  |
| $\boldsymbol{L_3}$ |  0  |   3   |   2   |   2   |   0   |   0   |   1   |   8   |
| $\boldsymbol{L_4}$ |  1  |  -5   |  -4   |  -3   |   0   |   0   |   0   |   0   |

In [2]:
M = np.array([
    [0,  2,  3,  1,  1,  0,  0,  5 ],
    [0,  4,  2,  2,  0,  1,  0,  11],
    [0,  3,  2,  2,  0,  0,  1,  8 ],
    [1, -5, -4, -3,  0,  0,  0,  0 ]
], dtype=np.float64)

### **Iteração 1**

Iniciamos selecionando a coluna de $x_1$ como **coluna pivô** por possuir custo com menor valor (-5). A partir disto, efetuamos a divisão *element-wise* dos elementos da coluna $b$ pelos elementos da coluna $x_1$, e selecionando o menor valor, obtemos a **linha pivô**.

In [3]:
b_col = 7
pivot_col = 1
div_pivot_b_cols = (M.T[b_col] / M.T[pivot_col])[:-1]
div_pivot_b_cols

array([2.5       , 2.75      , 2.66666667])

In [4]:
pivot_row = np.argmin(div_pivot_b_cols)
pivot_row

np.int64(0)

Logo, a linha pivô é $L_1$. Fazemos, então, a substituição $L_1 \rightarrow \frac{L_1}{2}$.

In [5]:
M[pivot_row] /= M[pivot_row][pivot_col]
M

array([[ 0. ,  1. ,  1.5,  0.5,  0.5,  0. ,  0. ,  2.5],
       [ 0. ,  4. ,  2. ,  2. ,  0. ,  1. ,  0. , 11. ],
       [ 0. ,  3. ,  2. ,  2. ,  0. ,  0. ,  1. ,  8. ],
       [ 1. , -5. , -4. , -3. ,  0. ,  0. ,  0. ,  0. ]])

A ideia agora é zerar todos os outros elementos da coluna $x_1$. Para tanto, fazemos as seguintes substituições:

* $L_2 \rightarrow L_2 - 4 L_1$
* $L_3 \rightarrow L_3 - 3 L_1$
* $L_4 \rightarrow L_4 + 5 L_1$

In [6]:
for i in range(M.shape[0]):
    if(i != pivot_row):
        M[i] -= M[i][pivot_col] * M[pivot_row]
M

array([[ 0. ,  1. ,  1.5,  0.5,  0.5,  0. ,  0. ,  2.5],
       [ 0. ,  0. , -4. ,  0. , -2. ,  1. ,  0. ,  1. ],
       [ 0. ,  0. , -2.5,  0.5, -1.5,  0. ,  1. ,  0.5],
       [ 1. ,  0. ,  3.5, -0.5,  2.5,  0. ,  0. , 12.5]])

### **Iteração 2**

Agora, a coluna pivô é a coluna de $x_3$. Efetuamos a divisão *element-wise* entre a coluna $b$ e a coluna $x_3$ para obter a linha pivô:

In [7]:
pivot_col = 3
div_pivot_b_cols = (M.T[b_col] / M.T[pivot_col])[:-1]
div_pivot_b_cols

/tmp/ipykernel_138353/1043914124.py:2: RuntimeWarning: divide by zero encountered in divide
  div_pivot_b_cols = (M.T[b_col] / M.T[pivot_col])[:-1]


array([ 5., inf,  1.])

In [8]:
pivot_row = np.argmin(div_pivot_b_cols)
pivot_row

np.int64(2)

A linha pivô agora é $L_3$. Fazemos, então, a substituição $L_3 \rightarrow \frac{L_3}{0.5}$.

In [9]:
M[pivot_row] /= M[pivot_row][pivot_col]
M

array([[ 0. ,  1. ,  1.5,  0.5,  0.5,  0. ,  0. ,  2.5],
       [ 0. ,  0. , -4. ,  0. , -2. ,  1. ,  0. ,  1. ],
       [ 0. ,  0. , -5. ,  1. , -3. ,  0. ,  2. ,  1. ],
       [ 1. ,  0. ,  3.5, -0.5,  2.5,  0. ,  0. , 12.5]])

Para zerar os outros elementos da coluna $x_3$:

* $L_1 \rightarrow L_1 - 0.5 L_3$
* $L_4 \rightarrow L_4 + 0.5 L_3$

In [10]:
for i in range(M.shape[0]):
    if(i != pivot_row):
        M[i] -= M[i][pivot_col] * M[pivot_row]
M

array([[ 0.,  1.,  4.,  0.,  2.,  0., -1.,  2.],
       [ 0.,  0., -4.,  0., -2.,  1.,  0.,  1.],
       [ 0.,  0., -5.,  1., -3.,  0.,  2.,  1.],
       [ 1.,  0.,  1.,  0.,  1.,  0.,  1., 13.]])

Repare que a linha de coeficientes de custo agora tem apenas valores positivos! Isto significa que chegamos ao final das iterações.

### **Resultado final**

Podemos ver, pelo resultado final, que as variáveis originais que fazem parte da base são $x_1$ e $x_3$, e que a resposta final é:

* $x_1 = 2$;
* $x_2 = 0$;
* $x_3 = 1$.

O ponto máximo da função é $Z = 13$.

## **3. Comprovação da solução por algoritmo**

In [11]:
A = np.array([
    [2, 3, 1],
    [4, 2, 2],
    [3, 2, 2]
])

b = [5, 11, 8]

coef = np.array([5, 4, 3]) * (-1)

var_bounds = np.array([[0, 0, 0], [np.inf, np.inf, np.inf]]).T

res = linprog(coef, A_ub=A, b_ub=b, bounds=var_bounds, method='highs')

In [12]:
print(f"Valor máximo da função objetivo: {-res.fun}")
print(f"x1 = {res.x[0]}, x2 = {res.x[1]}, x3 = {res.x[2]}")

Valor máximo da função objetivo: 13.0
x1 = 2.0, x2 = 0.0, x3 = 1.0
